In [2]:
from redis import StrictRedis

redis_forms = StrictRedis(host='localhost', port=7979, db=0)
redis_redir = StrictRedis(host='localhost', port=7979, db=1)

In [3]:
def calculate_redirect(source):
    result = redis_redir.get(source)
    if result is None:
        return source if type(source) is str else source.decode('utf-8')
    return calculate_redirect(result)

In [50]:
import pandas as pd


def query(surface_form):
    raw = redis_forms.hgetall(surface_form)
    if len(raw) == 0:
        return pd.DataFrame(columns=['entity', 'support', 'score'])
    
    out = []
    for label, score in raw.items():
        out.append({'entity': label.decode('utf-8'), 'support': int(score)})
    df_all = pd.DataFrame(out)
    df_all['score'] = df_all['support'] / df_all['support'].max()
    
    return df_all.sort_values(by='score', ascending=False).reset_index(drop=True)

df_out = query('United States')
df_out

,entity,support,score
0,United_States,208034,1.000000
1,United_States_men's_national_soccer_team,2095,0.010070
2,United_States_of_America,1415,0.006802
3,United_States_women's_national_soccer_team,890,0.004278
4,United_States_Soccer_Federation,757,0.003639
...,...,...,...
1965,2021_U.S._Figure_Skating_Championships,1,0.000005
1966,Lists_of_counties_in_the_United_States,1,0.000005
1967,LGBT_in_the_United_States,1,0.000005
1968,Swimming_at_the_1991_World_Aquatics_Championsh...,1,0.000005


In [53]:
def lookup(term, top_k=5, thr=0.01):
    df_temp = query(term)
#     display(df_temp)
    df_temp['entity'] = df_temp['entity'].apply(lambda x: calculate_redirect(x))
    if len(df_temp) == 0:
        return pd.DataFrame(columns=['entity', 'support', 'score'])
    df_final = df_temp.groupby('entity').sum()[['support']]
    df_final['score'] = df_final['support'] / df_final['support'].max()
    return df_final[df_final['score'] >= thr].sort_values(by='score', ascending=False)[:top_k]

In [64]:
print(lookup('US Copyright Office', top_k=10, thr=0.0001))

                                support  score
entity                                        
United_States_Copyright_Office       12    1.0


In [60]:
print(lookup('Bhilai', thr=0.01))

                            support     score
entity                                       
Bhilai                          182  1.000000
Electric_Loco_Shed,_Bhilai       15  0.082418
Bhilai_Steel_Plant                3  0.016484
IIT_Bhilai                        2  0.010989
